# 🚀 KR-BERT 감정 분류 모델 학습 (Google Colab)

이 노트북은 Google Colab의 **무료 GPU**를 사용하여 KR-BERT 기반 감정 분류 모델을 학습합니다.

**모델**: snunlp/KR-Medium (또는 snunlp/KR-BERT-char16424)  
**방식**: Hugging Face Trainer (자동 최적화, Early Stopping)  
**예상 시간**: GPU T4 기준 **2-3시간** (CPU 대비 8배 빠름)

## 📋 실행 순서
1. **런타임 설정**: 런타임 > 런타임 유형 변경 > **GPU** 선택
2. **셀 순서대로 실행**: Shift + Enter 또는 재생 버튼
3. **완료 후 다운로드**: 학습된 모델 파일 다운로드

## ✨ 특징
- ✅ **KR-BERT**: 한국어에 최적화된 BERT 모델
- ✅ **Hugging Face Trainer**: 안정적인 학습, 자동 체크포인팅
- ✅ **다양한 메트릭**: Accuracy, Precision, Recall, F1 Score
- ✅ **K-Fold CV**: 2-Fold Cross Validation 지원
- ✅ **Early Stopping**: 과적합 방지

## 1️⃣ GPU 확인 및 환경 설정

In [1]:
# GPU 사용 가능 여부 확인
import torch
print(f"🖥️ PyTorch 버전: {torch.__version__}")
print(f"🎮 CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"📊 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️ GPU를 사용할 수 없습니다. 런타임 > 런타임 유형 변경 > GPU를 선택하세요!")

🖥️ PyTorch 버전: 2.9.0+cpu
🎮 CUDA 사용 가능: False
⚠️ GPU를 사용할 수 없습니다. 런타임 > 런타임 유형 변경 > GPU를 선택하세요!


## 2️⃣ 필수 패키지 설치

In [ ]:
%%capture
# 필요한 패키지 설치
!pip install transformers>=4.30.0 datasets accelerate scikit-learn pandas numpy matplotlib seaborn

## 3️⃣ 데이터 업로드

**방법 1: 로컬 파일 업로드**
- 왼쪽 파일 아이콘 클릭
- `emotion_corpus_full.csv` 드래그 앤 드롭

**방법 2: Google Drive 연동** (아래 셀 실행)

In [ ]:
# 옵션 1: 로컬에서 직접 업로드
from google.colab import files
print("📂 통합된 데이터 파일을 업로드하세요:")
print("   - emotion_corpus_merged.csv (권장, 131K samples)")
print("   - 또는 emotion_corpus_full.csv (기존, 41K samples)")
uploaded = files.upload()

# 업로드된 파일 확인
import os
# 통합 데이터 우선 확인
if os.path.exists('emotion_corpus_merged.csv'):
    data_path = 'emotion_corpus_merged.csv'
    print(f"✅ 통합 데이터 업로드 완료: {data_path}")
elif os.path.exists('emotion_corpus_full.csv'):
    data_path = 'emotion_corpus_full.csv'
    print(f"✅ 기존 데이터 업로드 완료: {data_path}")
else:
    data_path = list(uploaded.keys())[0] if uploaded else None
    print(f"✅ 파일 업로드 완료: {data_path}")

if data_path and os.path.exists(data_path):
    print(f"📊 파일 크기: {os.path.getsize(data_path) / 1024 / 1024:.2f} MB")
else:
    print("❌ 파일을 찾을 수 없습니다. 다시 업로드하세요.")

In [ ]:
# 옵션 2: Google Drive 마운트 (선택사항)
# from google.colab import drive
# drive.mount('/content/drive')
# data_path = '/content/drive/MyDrive/emotion_corpus_full.csv'  # 경로 수정 필요

## 4️⃣ 데이터 확인

In [ ]:
import pandas as pd

# 데이터 로드
df = pd.read_csv(data_path)
print(f"📊 데이터 크기: {len(df):,} samples")
print(f"📋 컬럼: {df.columns.tolist()}")

# 감정 분포 확인
print("\n📈 감정 분포:")
if 'emotion' in df.columns:
    emotion_counts = df['emotion'].value_counts()
    for emotion, count in emotion_counts.items():
        percentage = count / len(df) * 100
        print(f"   - {emotion}: {count:,} ({percentage:.1f}%)")
elif 'label_id' in df.columns:
    label_counts = df['label_id'].value_counts().sort_index()
    emotion_names = ['joy', 'sad', 'anxiety', 'anger', 'neutral']
    for label_id, count in label_counts.items():
        emotion = emotion_names[label_id] if label_id < len(emotion_names) else f'label_{label_id}'
        percentage = count / len(df) * 100
        print(f"   - {emotion} (id={label_id}): {count:,} ({percentage:.1f}%)")

print("\n🔍 샘플 데이터:")
print(df.head(3).to_string())

## 5️⃣ 학습 코드 준비

In [ ]:
# 필요한 라이브러리 임포트
import numpy as np
import pandas as pd
import torch
from datetime import datetime

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.preprocessing import LabelEncoder

from torch.utils.data import Dataset

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)

import json
import gc
import warnings
warnings.filterwarnings('ignore')

# 시드 설정
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    
set_seed(42)
print("✅ 라이브러리 임포트 완료")

In [ ]:
# Hugging Face Trainer용 데이터셋 클래스
class EmotionDataset(Dataset):
    """Hugging Face Trainer용 데이터셋"""
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

print("✅ 데이터셋 클래스 정의 완료")

In [ ]:
# 평가 메트릭 계산 함수
def compute_metrics(p):
    """평가 메트릭 계산 (제공하신 코드 기반)"""
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall_micro = recall_score(y_true=labels, y_pred=pred, average="micro")
    recall_macro = recall_score(y_true=labels, y_pred=pred, average="macro")
    precision_micro = precision_score(y_true=labels, y_pred=pred, average="micro")
    precision_macro = precision_score(y_true=labels, y_pred=pred, average="macro")
    f1_macro = f1_score(y_true=labels, y_pred=pred, average="macro")

    return {
        "accuracy": accuracy,
        "recall_micro": recall_micro,
        "recall_macro": recall_macro,
        "precision_micro": precision_micro,
        "precision_macro": precision_macro,
        "f1_macro": f1_macro
    }

print("✅ 평가 메트릭 함수 정의 완료")

## 6️⃣ 학습 설정

In [ ]:
# 하이퍼파라미터 설정 (단일 분할 80/20 방식)
CONFIG = {
    'model_name': 'snunlp/KR-Medium',  # 또는 'snunlp/KR-BERT-char16424', 'klue/bert-base'
    'num_labels': 5,
    'max_length': 128,
    'batch_size': 64,  # GPU에서 가능
    'epochs': 10,  # 단일 분할이므로 에폭 늘림 (더 많이 학습)
    'learning_rate': 3e-5,
    'test_size': 0.2,  # 80/20 분할 (Train 80%, Val 20%)
    'early_stopping_patience': 6,
    'eval_steps': 500,
    'warmup_steps': 0,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

print("⚙️ 학습 설정 (단일 분할 80/20 방식):")
for key, value in CONFIG.items():
    print(f"   - {key}: {value}")

print("\n📊 데이터 분할 예상:")
total_samples = len(df) if 'df' in locals() else 131091
train_samples = int(total_samples * (1 - CONFIG['test_size']))
val_samples = total_samples - train_samples
print(f"   - Train: {train_samples:,} samples ({(1-CONFIG['test_size'])*100:.0f}%)")
print(f"   - Val: {val_samples:,} samples ({CONFIG['test_size']*100:.0f}%)")
print(f"   - 총합: {total_samples:,} samples")

if not torch.cuda.is_available():
    print("\n⚠️ GPU를 사용할 수 없습니다!")
    print("   런타임 > 런타임 유형 변경 > GPU를 선택하세요!")

In [ ]:
# 클래스 가중치 적용 모델 정의
from transformers import BertForSequenceClassification, AutoConfig
import torch.nn as nn

# (1) 클래스별 가중치 텐서 설정 (불균형 데이터 보정)
# 가중치 = 전체 샘플 수 / (클래스 수 * 각 클래스 샘플 수)
# 소수 클래스에 높은 가중치 부여
WEIGHTS = torch.tensor([3.01, 1.50, 1.18, 1.14, 0.48], dtype=torch.float32)

class WeightedLossBert(BertForSequenceClassification):
    """클래스 가중치를 적용한 CrossEntropyLoss를 사용하는 BERT 모델"""
    def __init__(self, config):
        super().__init__(config)
        
        # (2) Loss 함수 정의 시 class_weights 적용
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.loss_fct = nn.CrossEntropyLoss(weight=WEIGHTS.to(device))
        print(f"✅ 클래스 가중치 Loss 함수 초기화 완료 (device: {device})")

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, **kwargs):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        sequence_output = outputs[0]
        
        logits = self.classifier(sequence_output[:, 0, :])
        
        loss = None
        if labels is not None:
            # (3) 정의된 가중치 Loss 함수로 Loss 계산
            loss = self.loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            
        return (loss, logits) if loss is not None else (logits,)

print("✅ 클래스 가중치 적용 모델 클래스 정의 완료")
print(f"   - 가중치: {WEIGHTS.tolist()}")
print(f"   - 설명: [joy, sad, anxiety, anger, neutral]")
print(f"   - 소수 클래스(joy)에 높은 가중치, 다수 클래스(neutral)에 낮은 가중치")

In [ ]:
## 7️⃣ 단일 분할 (80/20) 학습 (Hugging Face Trainer)

## 7️⃣ K-Fold Cross Validation 학습 (Hugging Face Trainer)

In [ ]:
# 단일 분할 (80/20) 학습 실행
import time

# 데이터 준비
texts = df['text'].values
labels = df['label_id'].values

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

print(f"\n{'='*80}")
print(f"🚀 단일 분할 (80/20) 학습 시작")
print(f"   - 모델: {CONFIG['model_name']}")
print(f"   - 총 샘플: {len(texts):,}")
print(f"   - 배치 크기: {CONFIG['batch_size']}")
print(f"   - 에폭: {CONFIG['epochs']}")
print(f"{'='*80}\n")

start_time = time.time()

# Train/Val 분할 (80/20, Stratified)
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts,
    labels,
    test_size=CONFIG['test_size'],
    stratify=labels,  # 클래스 비율 유지
    random_state=42
)

print(f"📊 데이터 분할 완료:")
print(f"   - Train: {len(train_texts):,} samples ({(1-CONFIG['test_size'])*100:.0f}%)")
print(f"   - Val: {len(val_texts):,} samples ({CONFIG['test_size']*100:.0f}%)")

# 클래스 분포 확인
print(f"\n📈 Train 세트 클래스 분포:")
unique_train, counts_train = np.unique(train_labels, return_counts=True)
emotion_names = ['joy', 'sad', 'anxiety', 'anger', 'neutral']
for label_id, count in zip(unique_train, counts_train):
    emotion = emotion_names[label_id]
    percentage = count / len(train_labels) * 100
    print(f"   - {emotion} (id={label_id}): {count:,} ({percentage:.1f}%)")

# 토크나이징
print(f"\n🔤 토크나이징 중... (max_length={CONFIG['max_length']})")
train_encodings = tokenizer(
    train_texts.tolist(),
    padding=True,
    truncation=True,
    max_length=CONFIG['max_length']
)
val_encodings = tokenizer(
    val_texts.tolist(),
    padding=True,
    truncation=True,
    max_length=CONFIG['max_length']
)
print(f"✅ 토크나이징 완료")

# 데이터셋 생성
train_dataset = EmotionDataset(train_encodings, train_labels.tolist())
val_dataset = EmotionDataset(val_encodings, val_labels.tolist())

# 모델 로드 - 클래스 가중치 적용
print(f"\n🤖 가중치 적용 모델 로드: {CONFIG['model_name']}")
config = AutoConfig.from_pretrained(CONFIG['model_name'], num_labels=CONFIG['num_labels'])
model = WeightedLossBert.from_pretrained(CONFIG['model_name'], config=config)
print(f"✅ 클래스 가중치 적용 모델 로드 완료")
print(f"   - 가중치 Loss 함수로 불균형 데이터 보정")

# Training Arguments
output_dir = f"./single_split_{timestamp}"

training_args = TrainingArguments(
    output_dir=output_dir,
    eval_strategy="steps",
    eval_steps=CONFIG['eval_steps'],
    per_device_train_batch_size=CONFIG['batch_size'],
    per_device_eval_batch_size=CONFIG['batch_size'],
    num_train_epochs=CONFIG['epochs'],
    seed=42,
    load_best_model_at_end=True,
    learning_rate=CONFIG['learning_rate'],
    save_total_limit=1,
    logging_steps=100,
    save_strategy="steps",
    save_steps=CONFIG['eval_steps'],
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    warmup_steps=CONFIG['warmup_steps'],
    weight_decay=0.01,
    fp16=True,  # GPU 가속
    report_to="none",
)

# Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=CONFIG['early_stopping_patience'])]
    if CONFIG['early_stopping_patience'] > 0 else None,
)

# 학습
print(f"\n🚀 학습 시작...")
train_result = trainer.train()

# 평가
print(f"\n📊 최종 평가 중...")
eval_result = trainer.evaluate()

training_time = time.time() - start_time

print(f"\n{'='*60}")
print(f"📊 최종 결과:")
print(f"{'='*60}")
print(f"   - Accuracy: {eval_result['eval_accuracy']:.4f}")
print(f"   - F1 Macro: {eval_result['eval_f1_macro']:.4f}")
print(f"   - Precision Macro: {eval_result['eval_precision_macro']:.4f}")
print(f"   - Recall Macro: {eval_result['eval_recall_macro']:.4f}")
print(f"   - Loss: {eval_result['eval_loss']:.4f}")
print(f"   - 총 소요 시간: {training_time/60:.1f}분")
print(f"{'='*60}")

# 모델 저장
model_path = f"./best_model_{timestamp}"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)
print(f"\n💾 모델 저장: {model_path}")

# 결과 저장
result = {
    'timestamp': timestamp,
    'config': CONFIG,
    'accuracy': float(eval_result['eval_accuracy']),
    'f1_macro': float(eval_result['eval_f1_macro']),
    'precision_macro': float(eval_result['eval_precision_macro']),
    'recall_macro': float(eval_result['eval_recall_macro']),
    'loss': float(eval_result['eval_loss']),
    'training_time_minutes': training_time/60,
    'model_path': model_path,
    'train_samples': len(train_texts),
    'val_samples': len(val_texts)
}

# 메모리 정리
del model, trainer
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print(f"\n{'='*80}")
print(f"🎉 학습 완료!")
print(f"{'='*80}")

## 8️⃣ 결과 요약

In [ ]:
# 결과 요약
print("\n📊 학습 결과 요약:")
print(f"{'='*80}")
print(f"   ⏱️  학습 시간: {result['training_time_minutes']:.1f}분")
print(f"   📈 Accuracy: {result['accuracy']:.4f}")
print(f"   🎯 F1 Macro: {result['f1_macro']:.4f}")
print(f"   🔍 Precision: {result['precision_macro']:.4f}")
print(f"   🔄 Recall: {result['recall_macro']:.4f}")
print(f"   📉 Loss: {result['loss']:.4f}")
print(f"{'='*80}")

print(f"\n? 모델 정보:")
print(f"   - 모델 경로: {result['model_path']}")
print(f"   - Train 샘플: {result['train_samples']:,}")
print(f"   - Val 샘플: {result['val_samples']:,}")
print(f"   - 모델명: {CONFIG['model_name']}")

print(f"\n💡 성능 해석:")
if result['f1_macro'] >= 0.75:
    print(f"   ✅ 우수한 성능! (F1 ≥ 0.75)")
    print(f"   → 실제 챗봇 배포에 적합합니다.")
elif result['f1_macro'] >= 0.70:
    print(f"   ✅ 좋은 성능! (F1 ≥ 0.70)")
    print(f"   → 실전 사용 가능한 수준입니다.")
elif result['f1_macro'] >= 0.65:
    print(f"   ⚠️  괜찮은 성능 (F1 ≥ 0.65)")
    print(f"   → 추가 튜닝이 필요할 수 있습니다.")
else:
    print(f"   ⚠️  개선 필요 (F1 < 0.65)")
    print(f"   → 하이퍼파라미터 조정이나 더 많은 데이터가 필요합니다.")

print(f"\n🔄 K-Fold 방식과 비교:")
print(f"   - 단일 분할: 학습 시간 절반, 더 많은 데이터 사용")
print(f"   - K-Fold: 안정성 검증, 두 배 시간 소요")
print(f"   → 단일 분할로 {result['train_samples']:,}개 샘플로 학습 (K-Fold는 65K)")
print(f"   → 약 {result['train_samples'] - 65000:,}개 더 많은 데이터 활용!")

## 9️⃣ 모델 다운로드 (개선된 버전 - 다운로드 실패 방지)

In [ ]:
# 결과 JSON 저장
summary = {
    'timestamp': timestamp,
    'config': CONFIG,
    'result': result,
    'accuracy': float(result['accuracy']),
    'f1_macro': float(result['f1_macro']),
    'precision_macro': float(result['precision_macro']),
    'recall_macro': float(result['recall_macro']),
    'loss': float(result['loss']),
    'training_time_minutes': float(result['training_time_minutes']),
    'model_path': result['model_path'],
    'train_samples': result['train_samples'],
    'val_samples': result['val_samples']
}

with open('training_summary.json', 'w', encoding='utf-8') as f:
    json.dump(summary, f, ensure_ascii=False, indent=2)

print("✅ 결과 JSON 저장 완료: training_summary.json")
print("\n📄 JSON 내용:")
print(json.dumps(summary, ensure_ascii=False, indent=2))

In [ ]:
# ============================================
# 🎯 학습한 모델 다운로드 (개선된 버전 - 실패 방지)
# ============================================

from google.colab import files
import shutil
import json
import os

print("=" * 80)
print("📥 학습한 모델 다운로드 시작")
print("=" * 80)

try:
    # 1. JSON 파일에서 모델 정보 읽기
    if os.path.exists('training_summary.json'):
        with open('training_summary.json', 'r') as f:
            summary = json.load(f)
        
        model_folder = summary['model_path'].lstrip('./')  # './' 제거
        timestamp_str = summary['timestamp']
        f1_score = summary['f1_macro']
        accuracy = summary['accuracy']
    else:
        print("⚠️ training_summary.json을 찾을 수 없습니다.")
        print("💡 모델 폴더를 수동으로 찾습니다...\n")
        
        # 모든 모델 폴더 찾기
        all_models = [d for d in os.listdir('.') if os.path.isdir(d) and 'best_model' in d]
        if not all_models:
            raise FileNotFoundError("모델 폴더를 찾을 수 없습니다.")
        
        model_folder = all_models[0]
        timestamp_str = datetime.now().strftime('%Y%m%d_%H%M%S')
        f1_score = 0
        accuracy = 0
    
    print(f"✅ 모델 발견: {model_folder}")
    print(f"   - F1 Macro: {f1_score:.4f}")
    print(f"   - Accuracy: {accuracy:.4f}")
    
    # 2. 모델 폴더 존재 확인
    if not os.path.exists(model_folder):
        raise FileNotFoundError(f"모델 폴더를 찾을 수 없습니다: {model_folder}")
    
    # 3. 폴더 내 파일 확인
    model_files = os.listdir(model_folder)
    print(f"\n📦 모델 파일 목록 ({len(model_files)}개):")
    required_files = ['config.json', 'pytorch_model.bin', 'tokenizer_config.json']
    for req_file in required_files:
        if req_file in model_files:
            file_size = os.path.getsize(os.path.join(model_folder, req_file)) / 1024 / 1024
            print(f"   ✅ {req_file} ({file_size:.1f} MB)")
        else:
            print(f"   ⚠️ {req_file} (없음)")
    
    # 4. ZIP 압축
    zip_filename = f'best_model_{timestamp_str}'
    print(f"\n📦 압축 중: {model_folder}")
    print(f"   → {zip_filename}.zip")
    
    # 압축 시작
    shutil.make_archive(zip_filename, 'zip', model_folder)
    
    # 압축 파일 크기 확인
    if os.path.exists(f'{zip_filename}.zip'):
        zip_size = os.path.getsize(f'{zip_filename}.zip') / 1024 / 1024
        print(f"✅ 압축 완료! 크기: {zip_size:.1f} MB")
    else:
        raise FileNotFoundError("압축 파일이 생성되지 않았습니다.")
    
    # 5. 다운로드 (재시도 로직 포함)
    print(f"\n📥 다운로드 시작...")
    print("💡 브라우저에서 다운로드 허용을 클릭하세요!")
    
    try:
        files.download(f'{zip_filename}.zip')
        print("✅ 모델 ZIP 다운로드 완료!")
    except Exception as e:
        print(f"⚠️ 다운로드 오류: {e}")
        print("\n🔄 대안 방법:")
        print("   1. 왼쪽 파일 브라우저에서 수동 다운로드")
        print(f"   2. 파일명: {zip_filename}.zip")
        print("   3. 파일 우클릭 → Download")
    
    # 6. JSON 파일도 다운로드
    print(f"\n📥 학습 결과 JSON 다운로드...")
    try:
        files.download('training_summary.json')
        print("✅ JSON 다운로드 완료!")
    except Exception as e:
        print(f"⚠️ JSON 다운로드 실패: {e}")
    
    print("\n" + "=" * 80)
    print("✅ 다운로드 프로세스 완료!")
    print("=" * 80)
    print("\n💡 다운로드한 파일:")
    print(f"   1. {zip_filename}.zip - 학습된 모델 (약 {zip_size:.0f} MB)")
    print(f"   2. training_summary.json - 학습 결과 요약")
    
    print("\n📂 다음 단계:")
    print("   1. 다운로드한 파일 확인 (Downloads 폴더)")
    print("   2. 로컬 프로젝트로 이동:")
    print("      cd C:\\Users\\rlarl\\OneDrive\\Desktop\\chatbot")
    print("   3. ZIP 파일을 checkpoints_kfold\\ 폴더로 이동")
    print("   4. 압축 해제:")
    print(f"      Expand-Archive -Path '{zip_filename}.zip' -DestinationPath 'checkpoints_kfold\\'")
    print("   5. 모델 테스트:")
    print("      python test_model_integration.py")
    print("   6. 챗봇 실행:")
    print("      python fastapi_app/main.py")

except FileNotFoundError as e:
    print(f"\n❌ 오류: {e}")
    print("\n💡 해결 방법:")
    print("   1. 학습이 완료되었는지 확인")
    print("   2. 위의 학습 셀이 성공적으로 실행되었는지 확인")
    print("   3. 수동 다운로드:")
    print("      - 왼쪽 파일 브라우저 열기")
    print("      - 'best_model_*' 폴더 찾기")
    print("      - 폴더 우클릭 → Download")

except Exception as e:
    print(f"\n❌ 예상치 못한 오류: {e}")
    print(f"   오류 타입: {type(e).__name__}")
    print("\n💡 수동 다운로드 방법:")
    print("   1. 왼쪽 파일 아이콘 클릭 (📁)")
    print("   2. 'best_model_*' 폴더 찾기")
    print("   3. 폴더 우클릭 → Download")
    print("\n🔄 또는 다음 코드로 수동 압축:")
    print("   import shutil")
    print("   from google.colab import files")
    print("   model_folder = 'best_model_20251104_XXXXXX'  # 실제 폴더명")
    print("   shutil.make_archive(model_folder, 'zip', model_folder)")
    print("   files.download(f'{model_folder}.zip')")

finally:
    print("\n" + "=" * 80)
    print("📝 참고 사항")
    print("=" * 80)
    print("• 다운로드 실패 시: 파일 브라우저에서 수동 다운로드")
    print("• 브라우저 차단: 주소창 오른쪽의 차단 아이콘 클릭 → 허용")
    print("• 큰 파일: 다운로드에 10-30초 소요 가능")
    print("• ZIP 압축: 약 110MB 크기 (원본 폴더와 동일)")
    print("=" * 80)

## 🎉 학습 완료!

축하합니다! **KR-BERT 감정 분류 모델** 학습이 완료되었습니다.

### 📦 다운로드한 파일
1. **`best_model_{timestamp}.zip`** - 학습된 모델 (압축, 약 110MB)
2. **`training_summary.json`** - 학습 결과 요약

### 🔧 다음 단계

#### 1️⃣ Windows에서 압축 해제
```powershell
# PowerShell 실행
cd C:\Users\rlarl\OneDrive\Desktop\chatbot\checkpoints_kfold

# 다운로드한 ZIP 파일 이동 (Downloads 폴더에서)
Move-Item "$env:USERPROFILE\Downloads\best_model_*.zip" ".\"

# 압축 해제
Expand-Archive -Path "best_model_*.zip" -DestinationPath ".\"

# 파일 확인
dir best_model_*
```

#### 2️⃣ 모델 테스트
```powershell
cd C:\Users\rlarl\OneDrive\Desktop\chatbot
python test_model_integration.py
```

#### 3️⃣ 챗봇에 통합
```powershell
python fastapi_app/main.py
```

### 📊 단일 분할 (80/20) vs K-Fold 비교

| 항목 | 단일 분할 (80/20) | K-Fold (50/50) |
|------|------------------|----------------|
| **학습 데이터** | **104,872 samples** | 65,545 samples |
| **검증 데이터** | 26,219 samples | 65,546 samples |
| **예상 F1** | **0.720-0.760** ⭐ | 0.710 ± 0.002 |
| **학습 시간** | **90-120분** ⚡ | 180-240분 |
| **장점** | 더 많은 데이터 학습<br>시간 절약 | 안정성 검증<br>과적합 감지 |
| **단점** | 성능 변동 가능<br>검증 샘플 적음 | 시간 2배<br>학습 데이터 적음 |

### 🎯 예상 성능 (통합 데이터 131K samples 기준)

단일 분할 방식으로 **더 많은 데이터(104K)**를 학습하므로:

- **Accuracy**: 88-93% (K-Fold 대비 +1~3%p 향상 예상)
- **F1 Macro**: 0.72-0.76 (K-Fold 대비 +1~5%p 향상 예상)
- **Precision/Recall**: 0.71-0.75

### 💡 왜 단일 분할이 더 나을 수 있나요?

```
104,872 샘플로 학습 (vs K-Fold 65,545)
        ↓
약 39,000개 더 많은 데이터
        ↓
모델이 더 많은 패턴 학습
        ↓
검증 세트에서 더 높은 성능!
```

**하지만 주의**: 
- 검증 세트가 "운이 좋은" 샘플일 수 있음
- 실전 성능은 K-Fold와 비슷할 수도 있음
- 안정성은 K-Fold가 더 신뢰성 높음

### 🔄 추천 워크플로우

```
1. 현재: 단일 분할로 빠르게 학습 (90-120분)
         → F1 0.72-0.76 달성

2. 검증: 성능이 만족스럽다면 바로 배포
         또는 K-Fold로 안정성 재확인

3. 배포: FastAPI 챗봇에 통합하여 실전 테스트
```

### ⚠️ 다운로드 실패 시 대처법

#### 방법 1: 파일 브라우저에서 수동 다운로드
1. Colab 왼쪽 파일 아이콘(📁) 클릭
2. `best_model_YYYYMMDD_HHMMSS` 폴더 찾기
3. 폴더 우클릭 → **Download**

#### 방법 2: 개별 파일 다운로드
```python
from google.colab import files
import os

# 모델 폴더 찾기
models = [d for d in os.listdir('.') if 'best_model' in d]
model_folder = models[0]

# 중요 파일만 다운로드
important_files = [
    'config.json',
    'pytorch_model.bin',
    'tokenizer_config.json',
    'vocab.txt'
]

for file in important_files:
    file_path = os.path.join(model_folder, file)
    if os.path.exists(file_path):
        files.download(file_path)
```

#### 방법 3: Google Drive에 저장
```python
from google.colab import drive
import shutil

# Drive 마운트
drive.mount('/content/drive')

# 모델 복사
model_folder = 'best_model_20251104_XXXXXX'  # 실제 폴더명
drive_path = f'/content/drive/MyDrive/chatbot_models/{model_folder}'

shutil.copytree(model_folder, drive_path)
print(f"✅ Google Drive에 저장 완료: {drive_path}")
```

### 📚 추가 정보

- **모델 통합 가이드**: `MODEL_INTEGRATION_GUIDE.md` 참고
- **다운로드 가이드**: `COLAB_DOWNLOAD_GUIDE.md` 참고
- **API 문서**: http://localhost:8000/docs (서버 실행 후)

### ? 성공을 축하합니다!

단일 분할 방식으로:
- ✅ **시간 절약**: K-Fold 대비 절반 (90-120분)
- ✅ **더 많은 데이터**: 104K samples로 학습
- ✅ **더 높은 성능**: F1 0.72-0.76 예상
- ✅ **배포 준비**: 바로 사용 가능한 모델

이제 챗봇에 통합하여 실전 테스트를 시작하세요! 🎯